In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import system.utils.dynamic_properties as part_properties
import numpy as np
import casadi as ca

In [2]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, 3.142, 0.000, 0.000] #floating base mount
base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
kinematic_dict, K, P, L, D, C, g, qdd, joint_torque = alpha.build_model(root, tip, floating_base=True)
c_parms, m_params, I_params, vec_g, q, q_dot, q_dotdot, tau, base_pose, world_pose = kinematic_dict['parameters']
N = alpha.get_N
D_dot_2C = alpha.get_D_dot_2C
H = alpha.get_total_energy
H_dot = alpha.get_total_power

In [3]:
L.size(), K.size(), P.size(), D.size(), g.size()

((1, 1), (1, 1), (1, 1), (4, 4), (4, 1))

In [4]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               vec_g,
                               q,
                               q_dot,
                               base_pose,
                               world_pose ],[D , g, D_dot_2C, N, P])

In [5]:
joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
q_sample = np.array([5.23716914, 1.52230767, 0.08661294, 4.24328965])

robot_inertia_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f([5e-3 ,-1e-3 ,16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3],
    [0.341, 0.341, 0.341, 0.341],
    [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0],
    [0, 0, -9.81],
    q_sample,
    [0.0, 0.0, 0.0, 0.1],
    [0.190, 0.000, -0.120, 3.142, 0.000, 0.000],
    [0.0, 1.0, 0, 0, 0, 0])

robot_inertia_matrix, potential_energy

(DM(
 [[0.0529475, -3.10168e-05, 1.0251e-05, 0.00155671], 
  [-3.10168e-05, 0.0415245, -0.0185338, 2.28575e-05], 
  [1.0251e-05, -0.0185338, 0.0211285, 1.21337e-05], 
  [0.00155671, 2.28575e-05, 1.21337e-05, 0.0100089]]),
 DM(2.51301))

In [5]:
joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
q_sample = np.array([5.23716914, 1.52230767, 0.08661294, 4.24328965])

robot_inertia_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f([5e-3 ,-1e-3 ,16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3],
    [0.341, 0.341, 0.341, 0.341],
    [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0],
    [0, 0, -9.81],
    q_sample,
    [0.0, 0.0, 0.0, 0.1],
    [0.190, 0.000, -0.120, 3.142, 0.000, 0.000],
    [0.0, 1.0, 0, 0, 0, 0])

robot_inertia_matrix,potential_energy

(DM(
 [[0.0529475, -3.10168e-05, 1.0251e-05, 0.00155671], 
  [-3.10168e-05, 0.0415245, -0.0185338, 2.28575e-05], 
  [1.0251e-05, -0.0185338, 0.0211285, 1.21337e-05], 
  [0.00155671, 2.28575e-05, 1.21337e-05, 0.0100089]]),
 DM(2.51301))

In [7]:
for i in range(10000):
    joint_min = np.array([1.00, 0.01, 0.01, 0.01])
    joint_max = np.array([5.50, 3.40, 3.40, 5.70])
    q_sample = np.random.uniform(joint_min, joint_max)

    robot_inertia_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f([5e-3 ,-1e-3 ,16e-3,
        5e-3, -1e-3, 16e-3,
        5e-3, -1e-3, 16e-3,
        5e-3, -1e-3, 16e-3],
        [0.341, 0.341, 0.341, 0.341],
        [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0],
        [0, 0, -9.81],
        q_sample,
        [0.0, 0.0, 0.0, 0.1],
        [0.190, 0.000, -0.120, 3.142, 0.000, 0.000],
        [0.0, 1.0, 0, 0, 0, 0])

    checks = part_properties.is_spd_strict(robot_inertia_matrix) and part_properties.is_skew_symmetric(N_matrix) and np.allclose(D_dot_2C_matrix, N_matrix)
    if not checks:
        raise ValueError(f"check failed for q:{q_sample}")


ValueError: check failed for q:[3.14743594 1.56607959 2.8973664  0.64359439]